Importing needed modules:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as pyplot

import bc_utils as butils
from sklearn.datasets.samples_generator import make_blobs
from scipy.spatial import distance

Creating a test cluster:

In [ ]:
X, y = make_blobs(n_samples=300, centers=2, cluster_std=1.9, n_features=2)
df = pd.DataFrame(dict(x=X[:,0], y=X[:,1]))

Plotting test cluster for visualisation:

In [ ]:
df.head()

In [ ]:
plt = df.plot(x='x',y='y',kind='scatter', title='Test Cluster')

In [ ]:
def find_best_cluster_representative(cluster, similarity_measure):
    #input: np.array(of points)
    #returns a tuple
    
    min_sum = float("inf")
    best_repr_index = None
    for i in range(len(cluster)):
        distance_sum = 0;
        for point in cluster:
            distance_sum += similarity_measure(cluster[i], point)
            if (distance_sum > min_sum):
                break
        if(distance_sum < min_sum):
            min_sum = distance_sum
            best_repr_index = i
    
    return cluster[best_repr_index]

def delta_medoids_full(df, delta, similarity_measure):                                                         
    """Returns subset of input DataFrame, that is a good representation                                            
    of given data.                                                                                                 
                                                                                                                   
    This is a simplified delta-medoids algorithm. It finds out the                                                 
    representatives in one pass through the input data. Final representatives                                      
    depend on the ordering of input data.                                                                          
                                                                                                                   
    :param df: in data                                                                                             
    :type df: pandas.DataFrame                                                                                     
    :param delta: maximum distance of points to be considered similar                                              
    :type delta: float                                                                                             
    :param similarity_measure: similarity function to be used in algorithm                                         
    :type similarity_measure: scipy.spacial.distance                                                               
                                                                                                                   
    :Example:                                                                                                      
    >>> TODO"""
    t = 0
    representatives = {}
    representatives[t] = set()
    clusters = {}
    
    while True:
        print("\n=========== running for t = " + str(t) + "============")
        t = t + 1
        
        #================== RepAssign starts ==================                                                      
        for row in df.iterrows():
            dist = float("inf")
            represenative = None
        
            point = tuple(row[1])                                                                                      
                                                                                                                   
            for rep in clusters.keys():
                #finding the closest representative to current point
                if similarity_measure(point, rep) <= dist:
                    representative = rep
                    dist = similarity_measure(point, rep)                                                                                              
            if dist <= delta:
                clusters[representative] = np.vstack((clusters[representative], point))
            else:
                clusters[point] = np.array(point, ndmin=2)      
        #================== RepAssign ends ===================
        
        representatives[t] = set()
        for cluster in clusters.values():
            representative = find_best_cluster_representative(cluster, similarity_measure)
            print(representative)
            representatives[t].add(tuple(representative))
        #print(representatives[t])
        
        if representatives[t] == representatives[t-1]:
            break
                                                                                                                                                                                                                           
    return pd.DataFrame(list(representatives[t]), columns=df.columns.values)

In [ ]:
cluster_repr_old = butils.delta_medoids_one_shot(df, 1.5, distance.euclidean)
cluster_repr = delta_medoids_full(df, 1.5, distance.euclidean)

In [ ]:
cluster_repr.plot(x='x',y='y',kind='scatter', title='cluster_repr')
cluster_repr_old.plot(x='x',y='y',kind='scatter', title='cluster_repr_old')

In [ ]:
a = set([1, 2, 4, 5])
a = list(a)
a = pd.DataFrame(a)
a